In [13]:
import numpy as np
import pandas as pd
base_dir = '/data/coding/patent_qa/train/'
df_question = pd.read_json("/data/coding/patent_qa/train/questions.jsonl",lines=True)
quesion_vector = np.load('all_train_question_vectors.npy')
train_ocr_page_num_mapping = pd.read_csv('train_ocr_page_num_mapping.csv')
train_ocr_vectors = np.load('train_ocr_vectors.npy')
train_pdf_image_vectors = np.load("train_pdf_img_vectors.npy") # 前400个
train_pdf_image_page_num_mapping = pd.read_csv('train_pdf_img_page_num_mapping.csv') # 前400个

In [14]:
# 给定了图片的情况下，有图片本身，也需要召回对应的文本
def get_similar_text_embedding(base_dir,document_name,question_idx,top_k,pic_page_num):
    document_name = df_question.document[question_idx].split('.')[0]
    vec_idx = train_ocr_page_num_mapping[train_ocr_page_num_mapping['file_name']==document_name]['index'].values
    candidate_vec = train_ocr_vectors[vec_idx]
    query_vec = quesion_vector[question_idx]
    cos_sim = np.dot(candidate_vec, query_vec) / (np.linalg.norm(candidate_vec) * np.linalg.norm(query_vec))
    # 获取最相似的top_k个索引
    top_k_indices = np.argsort(cos_sim)[-(top_k+1):][::-1]
    retrived_idx = vec_idx[top_k_indices] # 最相近的top_k个
    retrived_page_num = train_pdf_image_page_num_mapping.loc[retrived_idx]['page_num'].to_list()
    retrived_page_num = [int(x) for x in retrived_page_num]
    # 如果pic_page_num>=0，返回的top_k个中不包含pic_page_num
    if pic_page_num >= 0:
        if pic_page_num in retrived_page_num:
            retrived_page_num.remove(pic_page_num)
    # 只返回前top_k个
    retrived_page_num = retrived_page_num[:top_k] # 最多返回top_k个
    # retrive_page_num排序
    retrived_page_num = sorted(retrived_page_num) # 按照page_num排序
    return retrived_page_num # 返回一个list,大小最大为top_k

def get_similar_image_embedding(base_dir,document_name,question_idx,top_k,pic_page_num):
    document_name = df_question.document[question_idx].split('.')[0]
    vec_idx = train_pdf_image_page_num_mapping[train_pdf_image_page_num_mapping['file_name']==document_name]['index'].values
    candidate_vec = train_pdf_image_vectors[vec_idx]
    query_vec = quesion_vector[question_idx]
    cos_sim = np.dot(candidate_vec, query_vec) / (np.linalg.norm(candidate_vec) * np.linalg.norm(query_vec))
    # 获取最相似的top_k个索引
    top_k_indices = np.argsort(cos_sim)[-(top_k+1):][::-1]
    retrived_idx = vec_idx[top_k_indices] # 最相近的top_k个
    retrived_page_num = train_pdf_image_page_num_mapping.loc[retrived_idx]['page_num'].to_list()
    retrived_page_num = [int(x) for x in retrived_page_num]
    # 如果pic_page_num>=0，返回的top_k个中不包含pic_page_num
    if pic_page_num >= 0:
        if pic_page_num in retrived_page_num:
            retrived_page_num.remove(pic_page_num)
    # 只返回前top_k个
    retrived_page_num = retrived_page_num[:top_k] # 最多返回top_k个
    # retrive_page_num排序
    retrived_page_num = sorted(retrived_page_num) # 按照page_num排序
    return retrived_page_num # 返回一个list,大小最大为top_k


def get_image_answer(document_name,question,options,question_idx,pic_page_num):
    question1 ="你是一个专利内容分析专家，请根据我提供的专利内容回答我的单选题。\n"
    question1 += "【我的问题】【"
    question1 += (question +"】\n")
    question1 += "【选项】【"
    question1 += (' '.join(options) + "】\n")
    question1 += ("专利内容为：\n")
    retrived_page_list = get_similar_image_embedding(base_dir,document_name,question_idx,2,pic_page_num)
    # 排序
    retrived_page_num = sorted(retrived_page_list)
    retrived_list = []
    for i in range(len(retrived_page_num)):
        image_file = base_dir + '/pdf_img/' + document_name.split('.')[0] + '/' + str(retrived_page_num[i]) +'.jpg'
        retrived_list.append(image_file)
    question2 = ("\n\n请你仔细分析专利内容后，回答我的单选题。回答最符合的选项字母，你的答案为：\n")
    if len(retrived_list)>1:
        query = ''
        query += question1
        query += '<image>'
        query += '<image>'
        query += question2
        images = []
        images.append(retrived_list[0])
        images.append(retrived_list[1])
        messages = [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": question1},
                    {
                        "type": "image",
                        "image": retrived_list[0],
                    },
                    {
                        "type": "image",
                        "image": retrived_list[1],
                    },
                    {"type": "text", "text": question2},
                ],
            }
        ]
    else:
        query = ''
        query += question1
        query += '<image>'
        query += question2
        images = []
        images.append(retrived_list[0])
        messages = [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": question1},
                    {
                        "type": "image",
                        "image": retrived_list[0],
                    },
                    {"type": "text", "text": question2},
                ],
            }
        ]
    # data = dict()
    # data['messages'] = messages
    # infer_requests = [InferRequest(**data)]
    # response = infer_batch(engine, infer_requests)
    return query,images


def get_mix_answer_img(document_name,pic_page_num,question,options,question_idx):
    question1 ="你是一个专利内容分析专家，请根据我提供的专利内容回答我的问题。\n"
    question1 += "【我的问题】【"
    question1 += (question +"】\n")
    question1 += "【选项】【"
    question1 += (' '.join(options) + "】\n")
    question1 += ("该问题直接指向的专利页内容为：\n")
    #retrived_list = get_similar_text_embedding(base_dir,document_name,question_idx,2)
    retrived_page_list = get_similar_image_embedding(base_dir,document_name,question_idx,2,pic_page_num)
    # 排序
    retrived_page_num = sorted(retrived_page_list)
    retrived_list = []
    for i in range(len(retrived_page_num)):
        image_file = base_dir + '/pdf_img/' + document_name.split('.')[0] + '/' + str(retrived_page_num[i]) +'.jpg'
        retrived_list.append(image_file)
    question2 = ("\n\n除了问题直接指向的专利页外，该专利其他相关内容为：\n")
    question3 = ("\n\n请你仔细分析专利内容后，回答我的单选题，回答最符合的选项字母，你的答案为：\n")
    if len(retrived_list)>1:
        query = ''
        query += question1
        query += '<image>'
        query += '<image>'
        query += '<image>'
        query += question2
        images = []
        images.append(base_dir + '/pdf_img/' + document_name.split('.')[0]+ '/' + str(pic_page_num) +'.jpg')
        images.append(retrived_list[0])
        images.append(retrived_list[1])
        messages = [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": question1},
                    {
                        "type": "image",
                        "image": base_dir + '/pdf_img/' + document_name.split('.')[0]+ '/' + str(pic_page_num) +'.jpg',
                    },
                    {"type": "text", "text": question2},
                    {
                        "type": "image",
                        "image": retrived_list[0],
                    },
                    {
                        "type": "image",
                        "image": retrived_list[1],
                    },
                    {"type": "text", "text": question3},
                ],
            }
        ]
    else:
        query = ''
        query += question1
        query += '<image>'
        query += question2
        images = []
        images.append(base_dir + '/pdf_img/' + document_name.split('.')[0]+ '/' + str(pic_page_num) +'.jpg')
        images.append(retrived_list[0])
        messages = [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": question1},
                    {
                        "type": "image",
                        "image": base_dir + '/pdf_img/' + document_name.split('.')[0]+ '/' + str(pic_page_num) +'.jpg',
                    },
                    {
                        "type": "image",
                        "image": retrived_list[0],
                    },
                    {"type": "text", "text": question2},
                ],
            }
        ]

    return query, images


In [15]:
def write_to_jsonl_img_train_set(x):
    tmp_dict = dict()
    tmp_dict['query'] = x['query']
    tmp_dict['document'] = x['document']
    tmp_dict['question_idx'] = int(x['question_idx'])
    tmp_dict['response'] = x['response']
    tmp_dict['images'] = x['images']
    with open('train_dataset_for_image_0615.jsonl', 'a', encoding='utf-8') as f:
        f.write(json.dumps(tmp_dict, ensure_ascii=False) + '\n')


import re
from tqdm import trange
import json
for i in trange(len(df_question)):
    question = df_question.loc[i,'question']
    document_name = df_question.loc[i,'document']
    options = df_question.loc[i,'options']
    true_answer = df_question.loc[i,'answer']
    full_question = question + ' '.join(options)
    answer = ''
    text_answer = ''
    image_answer = ''
    if "第" in question and "页" in question and "图": # 问题含有图片
        pic_page_num = re.findall(r"第(\d+)页", question)[0]
        pic_page_num = int(pic_page_num)
        query,images = get_mix_answer_img(document_name,pic_page_num,question,options,i)
        result_dict = dict()
        result_dict['question_idx'] = i
        result_dict['document'] = document_name
        result_dict['query'] = query
        result_dict['response'] = true_answer
        result_dict['images'] = images
        write_to_jsonl_img_train_set(result_dict)
    else:
        query,images = get_image_answer(document_name,question,options,i,-1) # 使用图像来回答
        result_dict = dict()
        result_dict['question_idx'] = i
        result_dict['document'] = document_name
        result_dict['query'] = query
        result_dict['response'] = true_answer
        result_dict['images'] = images
        write_to_jsonl_img_train_set(result_dict)
    #break



100%|██████████| 300/300 [00:00<00:00, 536.93it/s]
